In [ ]:
from tensorforce.environments import Environment
from tensorforce.agents import Agent
import tensorforce

In [2]:
class ReinforcementLearning:
        
    def __init__(self, ReinfocementLearning_Agent, Training_Eviroment, Demonstration_Eviroment) :
            
        # Reinforcement Agent Policy
        self.I_Am_ReinfocementLearning_Agent = ReinfocementLearning_Agent
        
        # Reinforcement Learning Eviroment    
        self.I_Am_Training_Eviroment = Training_Eviroment
        self.I_Am_Demonstration_Eviroment = Demonstration_Eviroment  
        
        # Traning Parameter 
        self.TrainingSampleTimes = 0
        self.TrainingRecord_Sampled_Trajectory = []
        self.TrainingRecord_Accumulated_Rewards = []
        self.TrainingRecord_Terminate_Steps = []
        
        # Demonstration Parameter
        self.DemonstrationSampleTimes = 0
        self.DemonstrationRecord_Accumulated_Rewards = []
        self.DemonstrationRecord_Terminate_Steps = []
 
    
    def AddRewardFunction(self, ObjectFunction_CaulculateReward):        
        self.I_Am_Reward_Function = ObjectFunction_CaulculateReward.I_Am_Reward_Function
    
    def Training_Agent(self, TrainingSampleTimes):
        
        self.TrainingSampleTimes = TrainingSampleTimes
        environment = self.I_Am_Training_Eviroment
        agent = self.I_Am_ReinfocementLearning_Agent

        for Sampling_Trajectory in range(self.TrainingSampleTimes):

            states = environment.reset()
            terminal = False
            
            Current_Steps = 0
            Final_Steps = Current_Steps
            Current_Trajectory_Record = []
            sum_rewards = 0.0
            
            print("========================================================================")
            print("Training Trajectory : {}".format(Sampling_Trajectory))
            print("========================================================================")
            
            while not terminal:          
                actions = agent.act(states=states)
                states, terminal, reward = environment.execute(actions=actions)
                reward = self.I_Am_Reward_Function(states, reward)
                agent.observe(terminal=terminal, reward=reward)

                print("Step:{}".format(Current_Steps))
                print("Action: {} Reward: {} States: {} ".format(actions, reward, states))

                Current_Steps = Current_Steps + 1   
                Current_Trajectory_Record.append({"Action": actions, "Reward": reward, "States": states})
                Final_Steps = Current_Steps
                sum_rewards = sum_rewards + reward
                
            print('Empirical Total Reward:', sum_rewards)
            self.TrainingRecord_Terminate_Steps.append(Final_Steps)
            self.TrainingRecord_Accumulated_Rewards.append(sum_rewards)
            self.TrainingRecord_Sampled_Trajectory.append(Current_Trajectory_Record)
            
        environment.close()
        
    def Demonstrating_Agent(self, DemonstrationSampleTimes):
        
        self.DemonstrationSampleTimes = DemonstrationSampleTimes
        environment = self.I_Am_Demonstration_Eviroment
        agent = self.I_Am_ReinfocementLearning_Agent
        
        for Sampling_Trajectory in range(self.DemonstrationSampleTimes):
            
            states = environment.reset()
            internals = agent.initial_internals()
            terminal = False
            
            Current_Steps = 0
            Final_Steps = Current_Steps
            sum_rewards = 0.0

            print("========================================================================")
            print("Demonstration Trajectory : {}".format(Sampling_Trajectory))
            print("========================================================================")            
            
            while not terminal:
                
                actions, internals = agent.act(states=states, internals=internals, evaluation=True)
                states, terminal, reward = environment.execute(actions=actions)
                reward = self.I_Am_Reward_Function(states, reward)
                
                print("Step:{}".format(Current_Steps))
                print("Action: {} Reward: {} States: {} ".format(actions, reward, states))
                
                Current_Steps = Current_Steps + 1
                Final_Steps = Current_Steps
                sum_rewards = sum_rewards + reward
                
            print('Empirical Total Reward:', sum_rewards)
            self.DemonstrationRecord_Terminate_Steps.append(Final_Steps)         
            self.DemonstrationRecord_Accumulated_Rewards.append(sum_rewards)
            
        environment.close()
        
    def Showing_Statistic_Graphics(self):
        
        print("Showing Graphical Pannel like matalib")
        print(self.TrainingRecord_Sampled_Trajectory)
        print(self.TrainingRecord_Accumulated_Rewards)
        print(self.TrainingRecord_Terminate_Steps)
        print(self.DemonstrationRecord_Accumulated_Rewards)
        print(self.DemonstrationRecord_Terminate_Steps)        


In [12]:
# GYM

# Discrete
# Acrobot-v1 # CartPole-v1

# Countinous
# MountainCarContinuous-v0 # Pendulum-v0

In [13]:
# Discrete

In [14]:
# Discrete
# CartPole-v1

# Objective Reward Function
class ObjectFunction_RewardCaulculation_CartPole:
    
    def I_Am_Reward_Function(states, reward):
        x, x_dot, theta, theta_dot = states
        reward_Position =  (10*(1 - abs(x)))**3
        reward_Angle =  (100*(0.1 - abs(theta)))**3
        I_Am_Official_Reward = reward_Position + reward_Angle  
        
        return I_Am_Official_Reward

In [79]:
# Dynamic Programing

In [ ]:
# Deep Q-Network

Training_Environment = Environment.create(environment='gym', level='CartPole-v1', max_episode_timesteps=500,  visualize=False)
Demo_Environment = Environment.create(environment='gym', level='CartPole-v1', max_episode_timesteps=500,  visualize=True)

layer_config = [
    dict(type='dense', size=4, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='dense', size=64, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='dense', size=64, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='flatten')]

agent_DQN = Agent.create(agent="dqn", network = layer_config, environment=Training_Environment, learning_rate=1e-2 , memory = 1000)    
    
ReinforcementLearning_Instance_CartPole_DQN = ReinforcementLearning(agent_DQN, Training_Environment, Demo_Environment)
ReinforcementLearning_Instance_CartPole_DQN.AddRewardFunction(ObjectFunction_RewardCaulculation_CartPole)

ReinforcementLearning_Instance_CartPole_DQN.Training_Agent(300)
ReinforcementLearning_Instance_CartPole_DQN.Demonstrating_Agent(10)
print(ReinforcementLearning_Instance_CartPole_DQN.TrainingRecord_Accumulated_Rewards)
print(ReinforcementLearning_Instance_CartPole_DQN.DemonstrationRecord_Accumulated_Rewards)
agent_DQN.close()

In [16]:
# Policy Graduim

In [ ]:
# Proximal Policy Optimization

Training_Environment = Environment.create(environment='gym', level='CartPole-v1', max_episode_timesteps=500,  visualize=False)
Demo_Environment = Environment.create(environment='gym', level='CartPole-v1', max_episode_timesteps=500,  visualize=True)

layer_config = [
    dict(type='dense', size=4, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='dense', size=64, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='dense', size=64, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='flatten')]

agent_PPO = Agent.create(agent="ppo", network = layer_config, environment=Training_Environment, batch_size=10, learning_rate=1e-3)    
    
ReinforcementLearning_Instance_CartPole_PPO = ReinforcementLearning(agent_PPO, Training_Environment, Demo_Environment)
ReinforcementLearning_Instance_CartPole_PPO.AddRewardFunction(ObjectFunction_RewardCaulculation_CartPole)

ReinforcementLearning_Instance_CartPole_PPO.Training_Agent(300)
ReinforcementLearning_Instance_CartPole_PPO.Demonstrating_Agent(10)
print(ReinforcementLearning_Instance_CartPole_PPO.TrainingRecord_Accumulated_Rewards)
print(ReinforcementLearning_Instance_CartPole_PPO.DemonstrationRecord_Accumulated_Rewards)
agent_PPO.close()

In [18]:
# Actor-Critic

In [ ]:
# Adventage Actor Critic

Training_Environment = Environment.create(environment='gym', level='CartPole-v1', max_episode_timesteps=500,  visualize=False)
Demo_Environment = Environment.create(environment='gym', level='CartPole-v1', max_episode_timesteps=500,  visualize=True)

layer_config = [
    dict(type='dense', size=4, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='dense', size=64, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='dense', size=64, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='flatten')]

agent_A2C = Agent.create(agent="a2c", network = layer_config, environment=Training_Environment, learning_rate=1e-3)
   
ReinforcementLearning_Instance_CartPole_A2C = ReinforcementLearning(agent_A2C, Training_Environment, Demo_Environment)
ReinforcementLearning_Instance_CartPole_A2C.AddRewardFunction(ObjectFunction_RewardCaulculation_CartPole)

ReinforcementLearning_Instance_CartPole_A2C.Training_Agent(300)
ReinforcementLearning_Instance_CartPole_A2C.Demonstrating_Agent(10)
print(ReinforcementLearning_Instance_CartPole_A2C.TrainingRecord_Accumulated_Rewards)
print(ReinforcementLearning_Instance_CartPole_A2C.DemonstrationRecord_Accumulated_Rewards)
agent_A2C.close()

In [20]:
# Discrete

In [21]:
# Discrete
# Acrobot-v1

# Objective Reward Function
class ObjectFunction_RewardCaulculation_Acrobot:
    
    def I_Am_Reward_Function(states, reward):
        
        I_Am_Official_Reward = -100 + abs(10*states[4]) + abs(10*states[5])**3
        
        return I_Am_Official_Reward

In [22]:
# Dynamic Programing

In [ ]:
# Deep Q-Network
Training_Environment = Environment.create(environment='gym', level='Acrobot-v1', max_episode_timesteps=500,  visualize=False)
Demo_Environment = Environment.create(environment='gym', level='Acrobot-v1', max_episode_timesteps=500,  visualize=True)

layer_config = [
    dict(type='dense', size=6, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='dense', size=64, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='dense', size=64, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='flatten')]
agent_DQN = Agent.create(agent="dqn", network= layer_config, environment=Training_Environment, learning_rate=1e-2 , memory = 1000)    

#agent_DQN = tensorforce.agents.DeepQNetwork(states = Training_Environment.states(), actions = Training_Environment.actions(), max_episode_timesteps = 500,network = layer_config, memory = 1000 , learning_rate=1e-3, horizon=0, discount=0.99)
#test_network = tensorforce.core.networks.LayeredNetwork( name = "lstm",  inputs_spec = {'type': 'float', 'shape': (4,)}, layers = layer_config)



ReinforcementLearning_Instance_Acrobot_DQN = ReinforcementLearning(agent_DQN, Training_Environment, Demo_Environment)
ReinforcementLearning_Instance_Acrobot_DQN.AddRewardFunction(ObjectFunction_RewardCaulculation_Acrobot)

ReinforcementLearning_Instance_Acrobot_DQN.Training_Agent(300)
ReinforcementLearning_Instance_Acrobot_DQN.Demonstrating_Agent(10)
print(ReinforcementLearning_Instance_Acrobot_DQN.TrainingRecord_Accumulated_Rewards)
print(ReinforcementLearning_Instance_Acrobot_DQN.DemonstrationRecord_Accumulated_Rewards)
agent_DQN.close()

In [ ]:
# Policy Graduim

In [ ]:
# Proximal Policy Optimization
Training_Environment = Environment.create(environment='gym', level='Acrobot-v1', max_episode_timesteps=500,  visualize=False)
Demo_Environment = Environment.create(environment='gym', level='Acrobot-v1', max_episode_timesteps=500,  visualize=True)

layer_config = [
    dict(type='dense', size=6, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='dense', size=64, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='dense', size=64, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='flatten')]
agent_PPO = Agent.create(agent="ppo", network = layer_config, environment=Training_Environment, batch_size=10, learning_rate=1e-3)    
    
ReinforcementLearning_Instance_Acrobot_PPO = ReinforcementLearning(agent_PPO, Training_Environment, Demo_Environment)
ReinforcementLearning_Instance_Acrobot_PPO.AddRewardFunction(ObjectFunction_RewardCaulculation_Acrobot)

ReinforcementLearning_Instance_Acrobot_PPO.Training_Agent(300)
ReinforcementLearning_Instance_Acrobot_PPO.Demonstrating_Agent(10)
print(ReinforcementLearning_Instance_Acrobot_PPO.TrainingRecord_Accumulated_Rewards)
print(ReinforcementLearning_Instance_Acrobot_PPO.DemonstrationRecord_Accumulated_Rewards)
agent_PPO.close()

Step:259
Action: 0 Reward: -85.79585152397335 States: [ 0.16011905  0.98709771 -0.64877126 -0.76098347 -0.88637214 -0.17479352] 
Step:260
Action: 1 Reward: 3459.918894749884 States: [ 0.44549804  0.89528291 -0.54372272 -0.83926492 -2.09145858  1.52391367] 
Step:261
Action: 2 Reward: 52050.49407148783 States: [ 0.83319202  0.55298378 -0.05469583 -0.99850306 -3.08209802  3.7353723 ] 
Step:262
Action: 1 Reward: 190235.0444711218 States: [ 0.99587086 -0.09078126  0.78635724 -0.61777204 -3.60529366  5.75191106] 
Step:263
Action: 1 Reward: 170880.01942870262 States: [ 0.70684664 -0.70736683  0.86983662  0.49333989 -3.12010369  5.54994528] 
Step:264
Action: 1 Reward: 45564.13627155438 States: [ 0.28846927 -0.95748915  0.14035904  0.99010067 -1.76453773  3.57384571] 
Step:265
Action: 2 Reward: 9660.19407167068 States: [ 0.0720633  -0.99740006 -0.41323518  0.91062434 -0.43798061  2.13675391] 
Step:266
Action: 2 Reward: 386.7091646316716 States: [ 0.11631976 -0.99321182 -0.65868666  0.75241736  

In [ ]:
# Actor-Critic

In [ ]:
# Adventage Actor Critic

Training_Environment = Environment.create(environment='gym', level='Acrobot-v1', max_episode_timesteps=500,  visualize=False)
Demo_Environment = Environment.create(environment='gym', level='Acrobot-v1', max_episode_timesteps=500,  visualize=True)

layer_config = [
    dict(type='dense', size=6, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='dense', size=64, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='dense', size=64, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='flatten')]

agent_A2C = Agent.create(agent="a2c", network = layer_config, environment=Training_Environment, learning_rate=1e-2)    
    
ReinforcementLearning_Instance_Acrobot_A2C = ReinforcementLearning(agent_A2C, Training_Environment, Demo_Environment)
ReinforcementLearning_Instance_Acrobot_A2C.AddRewardFunction(ObjectFunction_RewardCaulculation_Acrobot)

ReinforcementLearning_Instance_Acrobot_A2C.Training_Agent(300)
ReinforcementLearning_Instance_Acrobot_A2C.Demonstrating_Agent(10)
print(ReinforcementLearning_Instance_Acrobot_A2C.TrainingRecord_Accumulated_Rewards)
print(ReinforcementLearning_Instance_Acrobot_A2C.DemonstrationRecord_Accumulated_Rewards)
agent_A2C.close()

In [ ]:
# Countinous

In [3]:
# Countinous
# MountainCarContinuous-v0

# Objective Reward Function
class ObjectFunction_RewardCaulculation_MountainCarContinuous:
    
    def I_Am_Reward_Function(states, reward):

        I_Am_Official_Reward = reward 
        
        return I_Am_Official_Reward


In [4]:
# Policy Graduim

In [ ]:
# Proximal Policy Optimization

Training_Environment = Environment.create(environment='gym', level='MountainCarContinuous-v0', max_episode_timesteps=500,  visualize=False)
Demo_Environment = Environment.create(environment='gym', level='MountainCarContinuous-v0', max_episode_timesteps=500,  visualize=True)
agent_PPO = Agent.create(agent="ppo", environment=Training_Environment, batch_size=10, learning_rate=1e-3)    
    
ReinforcementLearning_Instance_MountainCarContinuous_PPO = ReinforcementLearning(agent_PPO, Training_Environment, Demo_Environment)
ReinforcementLearning_Instance_MountainCarContinuous_PPO.AddRewardFunction(ObjectFunction_RewardCaulculation_MountainCarContinuous)

ReinforcementLearning_Instance_MountainCarContinuous_PPO.Training_Agent(300)
ReinforcementLearning_Instance_MountainCarContinuous_PPO.Demonstrating_Agent(10)
print(ReinforcementLearning_Instance_MountainCarContinuous_PPO.TrainingRecord_Accumulated_Rewards)
print(ReinforcementLearning_Instance_MountainCarContinuous_PPO.DemonstrationRecord_Accumulated_Rewards)
agent_PPO.close()

In [ ]:
# Trust-Region Policy Optimization

Training_Environment = Environment.create(environment='gym', level='MountainCarContinuous-v0', max_episode_timesteps=500,  visualize=False)
Demo_Environment = Environment.create(envirotnment='gym', level='MountainCarContinuous-v0', max_episode_timesteps=500,  visualize=True)
agent_TRPO = Agent.create(agent="trpo", environment=Training_Environment, batch_size=10, learning_rate=1e-2)    
    
ReinforcementLearning_Instance_MountainCarContinuous_TRPO = ReinforcementLearning(agent_TRPO, Training_Environment, Demo_Environment)
ReinforcementLearning_Instance_MountainCarContinuous_TRPO.AddRewardFunction(ObjectFunction_RewardCaulculation_MountainCarContinuous)

ReinforcementLearning_Instance_MountainCarContinuous_TRPO.Training_Agent(300)
ReinforcementLearning_Instance_MountainCarContinuous_TRPO.Demonstrating_Agent(10)
print(ReinforcementLearning_Instance_MountainCarContinuous_TRPO.TrainingRecord_Accumulated_Rewards)
print(ReinforcementLearning_Instance_MountainCarContinuous_TRPO.DemonstrationRecord_Accumulated_Rewards)
agent_TRPO.close()

In [ ]:
# Actor-Critic

In [ ]:
#  Adventage Actor Critic

Training_Environment = Environment.create(environment='gym', level='MountainCarContinuous-v0', max_episode_timesteps=500,  visualize=False)
Demo_Environment = Environment.create(environment='gym', level='MountainCarContinuous-v0', max_episode_timesteps=500,  visualize=True)

layer_config = [
    dict(type='dense', size=2, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='dense', size=64, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='dense', size=64, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='dense', size=1, activation='relu')]

agent_A2C = Agent.create(agent="a2c", network = layer_config, environment=Training_Environment, learning_rate=1e-2)    
    
ReinforcementLearning_Instance_MountainCarContinuous_A2C = ReinforcementLearning(agent_A2C, Training_Environment, Demo_Environment)
ReinforcementLearning_Instance_MountainCarContinuous_A2C.AddRewardFunction(ObjectFunction_RewardCaulculation_MountainCarContinuous)

ReinforcementLearning_Instance_MountainCarContinuous_A2C.Training_Agent(300)
ReinforcementLearning_Instance_MountainCarContinuous_A2C.Demonstrating_Agent(10)
print(ReinforcementLearning_Instance_MountainCarContinuous_A2C.TrainingRecord_Accumulated_Rewards)
print(ReinforcementLearning_Instance_MountainCarContinuous_A2C.DemonstrationRecord_Accumulated_Rewards)
agent_A2C.close()

In [ ]:
# Countinous

In [ ]:
# Countinous
# Pendulum-v0

# Objective Reward Function
class ObjectFunction_RewardCaulculation_Pendulum:
    
    def I_Am_Reward_Function(states, reward):

        I_Am_Official_Reward = reward 
        
        return I_Am_Official_Reward

In [ ]:
# Policy Graduim

In [ ]:
# Proximal Policy Optimization

Training_Environment = Environment.create(environment='gym', level='Pendulum-v0', max_episode_timesteps=200,  visualize=False)
Demo_Environment = Environment.create(environment='gym', level='Pendulum-v0', max_episode_timesteps=200,  visualize=True)
agent_PPO = Agent.create(agent="ppo", environment=Training_Environment, batch_size=10, learning_rate=1e-3)    
    
ReinforcementLearning_Instance_Pendulum_PPO = ReinforcementLearning(agent_PPO, Training_Environment, Demo_Environment)
ReinforcementLearning_Instance_Pendulum_PPO.AddRewardFunction(ObjectFunction_RewardCaulculation_Pendulum)

ReinforcementLearning_Instance_Pendulum_PPO.Training_Agent(300)
ReinforcementLearning_Instance_Pendulum_PPO.Demonstrating_Agent(10)
print(ReinforcementLearning_Instance_Pendulum_PPO.TrainingRecord_Accumulated_Rewards)
print(ReinforcementLearning_Instance_Pendulum_PPO.DemonstrationRecord_Accumulated_Rewards)
agent_PPO.close()

In [ ]:
# Trust-Region Policy Optimization

Training_Environment = Environment.create(environment='gym', level='Pendulum-v0', max_episode_timesteps=200,  visualize=False)
Demo_Environment = Environment.create(environment='gym', level='Pendulum-v0', max_episode_timesteps=200,  visualize=True)
agent_TRPO = Agent.create(agent="trpo", environment=Training_Environment, batch_size=10, learning_rate=1e-2)    
    
ReinforcementLearning_Instance_Pendulum_TRPO = ReinforcementLearning(agent_TRPO, Training_Environment, Demo_Environment)
ReinforcementLearning_Instance_Pendulum_TRPO.AddRewardFunction(ObjectFunction_RewardCaulculation_Pendulum)

ReinforcementLearning_Instance_Pendulum_TRPO.Training_Agent(300)
ReinforcementLearning_Instance_Pendulum_TRPO.Demonstrating_Agent(10)
print(ReinforcementLearning_Instance_Pendulum_TRPO.TrainingRecord_Accumulated_Rewards)
print(ReinforcementLearning_Instance_Pendulum_TRPO.DemonstrationRecord_Accumulated_Rewards)
agent_TRPO.close()

In [ ]:
# Actor-Critic

In [ ]:
#  Adventage Actor Critic

Training_Environment = Environment.create(environment='gym', level='Pendulum-v0', max_episode_timesteps=200,  visualize=False)
Demo_Environment = Environment.create(environment='gym', level='Pendulum-v0', max_episode_timesteps=200,  visualize=True)

layer_config = [
    dict(type='dense', size=2, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='dense', size=64, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='dense', size=64, activation='relu'),
    dict(type='dense', size=128, activation='relu'),
    dict(type='dense', size=1, activation='relu')]

agent_A2C = Agent.create(agent="a2c", network = layer_config, environment=Training_Environment, learning_rate=1e-2)    
    
ReinforcementLearning_Instance_Pendulum_A2C = ReinforcementLearning(agent_A2C, Training_Environment, Demo_Environment)
ReinforcementLearning_Instance_Pendulum_A2C.AddRewardFunction(ObjectFunction_RewardCaulculation_Pendulum)

ReinforcementLearning_Instance_Pendulum_A2C.Training_Agent(300)
ReinforcementLearning_Instance_Pendulum_A2C.Demonstrating_Agent(10)
print(ReinforcementLearning_Instance_Pendulum_A2C.TrainingRecord_Accumulated_Rewards)
print(ReinforcementLearning_Instance_Pendulum_A2C.DemonstrationRecord_Accumulated_Rewards)
agent_A2C.close()

In [ ]:
# openai_retro 

#the 128 sine-dot by Anthrox
#Sega Tween by Ben Ryves
#Happy 10! by Blind IO
#512-Colour Test Demo by Chris Covell
#Dekadrive by Dekadence
#Automaton by Derek Ledbetter
#Fire by dox
#FamiCON intro by dr88
#Airstriker by Electrokinesis
#Lost Marbles by Vantage

In [ ]:
class ObjectFunction_RewardCaulculation_openai_retro:
    
    def I_Am_Reward_Function(states, reward):

        I_Am_Official_Reward = reward 
        
        return I_Am_Official_Reward

In [ ]:
# Dynamic Programing

In [ ]:
# Deep Q-Network

Training_Environment = Environment.create(environment='openai_retro', level='Airstriker-Genesis', max_episode_timesteps=500,  visualize=False)
agent_DQN = Agent.create(agent="dqn", environment=Training_Environment, batch_size=10, learning_rate=1e-3 , memory = 10000)    
    
ReinforcementLearning_Instance_openai_retro_DQN = ReinforcementLearning(agent_DQN, Training_Environment, Training_Environment)
ReinforcementLearning_Instance_openai_retro_DQN.AddRewardFunction(ObjectFunction_RewardCaulculation_openai_retro)
ReinforcementLearning_Instance_openai_retro_DQN.Training_Agent(500)
print(ReinforcementLearning_Instance_HandManipulatePen_DQN.TrainingRecord_Accumulated_Rewards)


Demo_Environment = Environment.create(environment='openai_retro', level='Airstriker-Genesis', max_episode_timesteps=500,  visualize=True)
ReinforcementLearning_Instance_openai_retro_DQN = ReinforcementLearning(agent_DQN, Demo_Environment, Demo_Environment)
ReinforcementLearning_Instance_openai_retro_DQN.AddRewardFunction(ObjectFunction_RewardCaulculation_openai_retro)
ReinforcementLearning_Instance_openai_retro_DQN.Demonstrating_Agent(10)
print(ReinforcementLearning_Instance_HandManipulatePen_DQN.DemonstrationRecord_Accumulated_Rewards)

agent_DQN.close()

In [ ]:
# Policy Graduim

In [ ]:
# Proximal Policy Optimization

Training_Environment = Environment.create(environment='openai_retro', level='Airstriker-Genesis', max_episode_timesteps=500,  visualize=False)
agent_PPO = Agent.create(agent="ppo", environment=Training_Environment, batch_size=10, learning_rate=1e-3)   
    
ReinforcementLearning_Instance_openai_retro_PPO = ReinforcementLearning(agent_DQN, Training_Environment, Training_Environment)
ReinforcementLearning_Instance_openai_retro_PPO.AddRewardFunction(ObjectFunction_RewardCaulculation_openai_retro)
ReinforcementLearning_Instance_openai_retro_PPO.Training_Agent(3000)
print(ReinforcementLearning_Instance_openai_retro_PPO.TrainingRecord_Accumulated_Rewards)

Demo_Environment = Environment.create(environment='openai_retro', level='Airstriker-Genesis', max_episode_timesteps=500,  visualize=True)
ReinforcementLearning_Instance_openai_retro_PPO = ReinforcementLearning(agent_DQN, Demo_Environment, Demo_Environment)
ReinforcementLearning_Instance_openai_retro_PPO.AddRewardFunction(ObjectFunction_RewardCaulculation_openai_retro)
ReinforcementLearning_Instance_openai_retro_PPO.Demonstrating_Agent(10)
print(ReinforcementLearning_Instance_openai_retro_PPO.DemonstrationRecord_Accumulated_Rewards)

agent_PPO.close()

In [ ]:
# Actor Critic

In [ ]:
# Adventage Actor Critic

Training_Environment = Environment.create(environment='openai_retro', level='Airstriker-Genesis', max_episode_timesteps=500,  visualize=False)
agent_A2C = Agent.create(agent="a2c", environment=Training_Environment, batch_size=10, learning_rate=1e-3)   
    
ReinforcementLearning_Instance_openai_retro_A2C = ReinforcementLearning(agent_A2C, Training_Environment, Training_Environment)
ReinforcementLearning_Instance_openai_retro_A2C.AddRewardFunction(ObjectFunction_RewardCaulculation_openai_retro)
ReinforcementLearning_Instance_openai_retro_A2C.Training_Agent(3000)
print(ReinforcementLearning_Instance_openai_retro_A2C.TrainingRecord_Accumulated_Rewards)

Demo_Environment = Environment.create(environment='openai_retro', level='Airstriker-Genesis', max_episode_timesteps=500,  visualize=True)
ReinforcementLearning_Instance_openai_retro_A2C = ReinforcementLearning(agent_DQN, Demo_Environment, Demo_Environment)
ReinforcementLearning_Instance_openai_retro_A2C.AddRewardFunction(ObjectFunction_RewardCaulculation_openai_retro)
ReinforcementLearning_Instance_openai_retro_A2C.Demonstrating_Agent(10)
print(ReinforcementLearning_Instance_openai_retro_A2C.DemonstrationRecord_Accumulated_Rewards)

agent_A2C.close()